# Dueling DQN - Interactive Exercise

Welcome! In this notebook, you will implement **Dueling DQN**, an architectural improvement over standard DQN and Double DQN.

## What is Dueling DQN?

Dueling DQN (Wang et al., 2016) changes the **network architecture** to separately estimate:
- **State Value** V(s): How good is this state?
- **Advantage** A(s,a): How much better is each action?

Then combines them to get Q(s,a)!

## The Key Insight

In many states, the **value of the state** matters more than **which specific action** you take.

**Example** (Driving):
- **Bad state**: About to crash → V(s) very low, all actions are bad
- **Good state**: Open highway → V(s) high, most actions are similar
- **Critical state**: Intersection → V(s) moderate, action choice matters!

Dueling DQN learns V(s) and A(s,a) separately, which helps in states where action choice doesn't matter much.

## Architecture Comparison

**Standard DQN**:
```
State → FC → FC → Q(s,a₁), Q(s,a₂), ..., Q(s,aₙ)
```

**Dueling DQN**:
```
State → Shared FC → ┬─→ Value Stream → V(s)
                     └─→ Advantage Stream → A(s,a₁), A(s,a₂), ..., A(s,aₙ)
                     
Q(s,a) = V(s) + (A(s,a) - mean(A(s,:)))
```

## Key Differences from DQN/Double DQN

| Aspect | DQN/Double DQN | Dueling DQN |
|--------|----------------|-------------|
| Architecture | Single stream | **Dual stream** (V + A) |
| Q-value Estimate | Direct | **Decomposed**: V(s) + A(s,a) |
| Learning | All actions together | **Separate value and advantage** |
| Performance | Good | **Better** (especially in sparse rewards) |
| Generalization | Moderate | **Better** (V(s) shared across actions) |

## Learning Objectives

By the end of this notebook, you will:
- Understand the dueling architecture
- Implement value and advantage streams
- Combine them correctly to get Q-values
- See why this decomposition helps learning
- Compare Dueling DQN with standard DQN

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gymnasium as gym
import matplotlib.pyplot as plt
from collections import deque, namedtuple
import random
from dueling_dqn_tests import *

In [ ]:
# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

## The Environment: CartPole

We'll use CartPole-v1 to compare Dueling DQN with standard DQN.

In [ ]:
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

print(f"State dimension: {state_dim}")
print(f"Action dimension: {action_dim}")

## Exercise 1: Dueling Q-Network Architecture

The dueling architecture has three parts:
1. **Shared layers**: Common feature extraction
2. **Value stream**: Estimates V(s) → single output
3. **Advantage stream**: Estimates A(s,a) → one output per action

**Combining Formula** (with mean subtraction for identifiability):
$$Q(s,a) = V(s) + \left(A(s,a) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s,a')\right)$$

Why subtract mean? To make V and A **identifiable** (unique). Without it, you could add a constant to V and subtract it from A without changing Q!

**Task**: Implement the Dueling Q-Network.

In [ ]:
# GRADED FUNCTION: DuelingQNetwork

class DuelingQNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        """
        Dueling Q-Network with separate value and advantage streams.
        
        Arguments:
        state_dim -- dimension of state space
        action_dim -- dimension of action space  
        hidden_dim -- number of hidden units
        """
        super(DuelingQNetwork, self).__init__()
        
        # (approx. 6-7 lines)
        # 1. Shared layers:
        #    fc1: state_dim -> hidden_dim
        # 2. Value stream:
        #    value_fc: hidden_dim -> hidden_dim//2
        #    value_out: hidden_dim//2 -> 1 (single value)
        # 3. Advantage stream:
        #    advantage_fc: hidden_dim -> hidden_dim//2
        #    advantage_out: hidden_dim//2 -> action_dim
        
        # YOUR CODE STARTS HERE
        
        
        # YOUR CODE ENDS HERE
    
    def forward(self, state):
        """
        Forward pass through dueling architecture.
        
        Arguments:
        state -- state tensor
        
        Returns:
        q_values -- combined Q-values for each action
        """
        # (approx. 12-15 lines)
        # 1. Shared features:
        #    x = F.relu(self.fc1(state))
        # 2. Value stream:
        #    v = F.relu(self.value_fc(x))
        #    value = self.value_out(v)  # Shape: (batch, 1)
        # 3. Advantage stream:
        #    a = F.relu(self.advantage_fc(x))
        #    advantages = self.advantage_out(a)  # Shape: (batch, action_dim)
        # 4. Combine with mean subtraction:
        #    q_values = value + (advantages - advantages.mean(dim=1, keepdim=True))
        #    This ensures Q(s,a) = V(s) + A(s,a) - mean(A(s,:))
        
        # YOUR CODE STARTS HERE
        
        
        # YOUR CODE ENDS HERE
        
        return q_values

In [ ]:
# Test your implementation
dueling_qnetwork_test(DuelingQNetwork)

## Exercise 2: Replay Buffer

Same as DQN and Double DQN - we need experience replay.

**Task**: Implement the replay buffer.

In [ ]:
# GRADED FUNCTION: DuelingReplayBuffer

Transition = namedtuple('Transition', ['state', 'action', 'reward', 'next_state', 'done'])

class DuelingReplayBuffer:
    def __init__(self, capacity=10000):
        """Experience replay buffer."""
        # (approx. 1 line)
        
        # YOUR CODE STARTS HERE
        
        # YOUR CODE ENDS HERE
    
    def push(self, state, action, reward, next_state, done):
        """Store a transition."""
        # (approx. 1 line)
        
        # YOUR CODE STARTS HERE
        
        # YOUR CODE ENDS HERE
    
    def sample(self, batch_size):
        """Sample a batch of transitions."""
        # (approx. 1 line)
        
        # YOUR CODE STARTS HERE
        
        # YOUR CODE ENDS HERE
        
        return batch
    
    def __len__(self):
        return len(self.buffer)

In [ ]:
# Test your implementation
dueling_replay_buffer_test(DuelingReplayBuffer)

## Exercise 3: Compute Dueling DQN Loss

The loss function can use either:
- **Standard DQN**: max over target network
- **Double DQN**: action selection from online, evaluation from target

We'll use **Double DQN style** (it's better!).

The key difference from before: The network architecture is dueling, but the loss is the same!

**Task**: Implement the loss function for Dueling DQN.

In [ ]:
# GRADED FUNCTION: compute_dueling_dqn_loss

def compute_dueling_dqn_loss(batch, online_net, target_net, gamma=0.99):
    """
    Compute Dueling DQN loss (using Double DQN approach).
    
    Arguments:
    batch -- list of Transition namedtuples
    online_net -- online Dueling Q-network
    target_net -- target Dueling Q-network
    gamma -- discount factor
    
    Returns:
    loss -- scalar loss value
    """
    # (approx. 15-18 lines)
    # This is the same as Double DQN loss, but using DuelingQNetwork!
    # 1. Unpack batch into tensors
    # 2. Get current Q-values: online_net(states).gather(1, actions)
    # 3. Double DQN target:
    #    a. Select actions: next_actions = online_net(next_states).argmax(1, keepdim=True)
    #    b. Evaluate: next_q = target_net(next_states).gather(1, next_actions)
    #    c. Target: reward + gamma * next_q * (1 - done)
    # 4. Compute loss: F.mse_loss or F.smooth_l1_loss
    
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    return loss

In [ ]:
# Test your implementation
compute_dueling_dqn_loss_test(compute_dueling_dqn_loss, DuelingQNetwork, DuelingReplayBuffer)

## Exercise 4: Update Target Network

Same as Double DQN - periodically copy weights.

**Task**: Implement target network update.

In [ ]:
# GRADED FUNCTION: update_dueling_target

def update_dueling_target(online_net, target_net):
    """
    Copy weights from online to target network.
    
    Arguments:
    online_net -- online Dueling Q-network
    target_net -- target Dueling Q-network
    """
    # (approx. 1 line)
    
    # YOUR CODE STARTS HERE
    
    # YOUR CODE ENDS HERE

In [ ]:
# Test your implementation
update_dueling_target_test(update_dueling_target, DuelingQNetwork)

## Exercise 5: Train Dueling DQN

Training loop is the same as Double DQN, but using the dueling architecture!

**Task**: Implement the complete training algorithm.

In [ ]:
# GRADED FUNCTION: train_dueling_dqn

def train_dueling_dqn(env, n_episodes=500, gamma=0.99, epsilon_start=1.0,
                      epsilon_end=0.01, epsilon_decay=0.995, lr=1e-3,
                      batch_size=64, target_update_freq=10):
    """
    Train Dueling DQN on the environment.
    
    Arguments:
    env -- Gym environment
    n_episodes -- number of episodes
    gamma -- discount factor
    epsilon_start -- initial epsilon
    epsilon_end -- minimum epsilon
    epsilon_decay -- epsilon decay rate
    lr -- learning rate
    batch_size -- batch size for training
    target_update_freq -- update target network every N episodes
    
    Returns:
    episode_rewards -- list of rewards per episode
    online_net -- trained network
    """
    # (approx. 35-40 lines)
    # Same structure as Double DQN training:
    # 1. Initialize online and target DuelingQNetwork
    # 2. Initialize optimizer and replay buffer
    # 3. For each episode:
    #    a. Epsilon-greedy action selection
    #    b. Store transitions
    #    c. Sample and train when buffer is ready
    #    d. Update target network periodically
    #    e. Decay epsilon
    #    f. Track rewards
    
    # YOUR CODE STARTS HERE
    
    
    # YOUR CODE ENDS HERE
    
    return episode_rewards, online_net

In [ ]:
# Test your implementation
train_dueling_dqn_test(train_dueling_dqn)

## Full Training Run

Let's train Dueling DQN and see the benefits!

In [ ]:
# Train Dueling DQN
episode_rewards, trained_net = train_dueling_dqn(
    env,
    n_episodes=500,
    gamma=0.99,
    epsilon_decay=0.995,
    target_update_freq=10
)

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(episode_rewards, alpha=0.6)
plt.plot(np.convolve(episode_rewards, np.ones(50)/50, mode='valid'), linewidth=2)
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Dueling DQN Training')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
window = 100
if len(episode_rewards) >= window:
    moving_avg = np.convolve(episode_rewards, np.ones(window)/window, mode='valid')
    plt.plot(moving_avg)
    plt.axhline(y=475, color='r', linestyle='--', label='Solved')
    plt.xlabel('Episode')
    plt.ylabel(f'Avg ({window} ep)')
    plt.title('Moving Average')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

if len(episode_rewards) >= 100:
    final = np.mean(episode_rewards[-100:])
    print(f"\n{'🎉 Solved!' if final >= 475 else '📊 Complete'} Final: {final:.2f}")

## Why Dueling DQN Works Better

### 1. **Better Value Generalization**
- V(s) is shared across all actions
- Learning about state value helps estimate Q for all actions
- Especially useful when some actions don't matter much

### 2. **More Frequent Value Updates**
- Every transition updates V(s), regardless of which action was taken
- In standard DQN, only Q(s, a_taken) is updated
- Dueling updates V(s) for **every** experience!

### 3. **Better in Sparse Reward Environments**
- Can learn V(s) even when actions don't lead to rewards
- Helps with credit assignment

### 4. **Identifiability Through Mean Subtraction**
- Without mean subtraction: Q = V + A is not unique
- With mean subtraction: Forces V to represent state value, A to represent relative advantage

## Comparison: DQN vs Double DQN vs Dueling DQN

| Aspect | DQN | Double DQN | Dueling DQN |
|--------|-----|------------|-------------|
| Architecture | Single stream | Single stream | **Dual stream** |
| Q Estimation | Direct | Decoupled selection/eval | **Decomposed V + A** |
| Overestimation | High | Reduced | Reduced |
| Value Learning | Slow | Moderate | **Fast** (V shared) |
| Sparse Rewards | Moderate | Moderate | **Good** |
| Performance | Good | Better | **Best** |

## Best Practice: Combine Them!

**Dueling Double DQN** = Dueling architecture + Double DQN loss

This is what we implemented here! It combines:
- Dueling architecture → better value learning
- Double DQN loss → reduced overestimation

Result: **State-of-the-art value-based RL** (before Rainbow)!

## Visualizing the Dueling Architecture

Let's inspect what the network learned!

In [ ]:
# Get a sample state
state, _ = env.reset()
state_tensor = torch.FloatTensor(state).unsqueeze(0)

# Forward pass through dueling network
with torch.no_grad():
    # Get Q-values
    q_values = trained_net(state_tensor)
    
    # Also compute V and A separately for visualization
    x = F.relu(trained_net.fc1(state_tensor))
    
    v = F.relu(trained_net.value_fc(x))
    value = trained_net.value_out(v)
    
    a = F.relu(trained_net.advantage_fc(x))
    advantages = trained_net.advantage_out(a)

print("\nDueling Architecture Decomposition:")
print(f"State Value V(s): {value.item():.4f}")
print(f"Advantages A(s,a): {advantages.numpy()[0]}")
print(f"Final Q-values Q(s,a): {q_values.numpy()[0]}")
print(f"\nVerify: Q = V + (A - mean(A))")
print(f"  V + (A - mean(A)) = {(value + (advantages - advantages.mean())).numpy()[0]}")

## Congratulations!

You've successfully implemented Dueling DQN! You now understand:
- ✅ The dueling architecture with value and advantage streams
- ✅ Why decomposing Q into V and A helps learning
- ✅ The importance of mean subtraction for identifiability
- ✅ How to combine Dueling with Double DQN
- ✅ When and why Dueling DQN outperforms standard DQN

**You've now mastered the core DQN family of algorithms!**

**Next Steps**:
- Implement **Rainbow DQN** (combines 6+ improvements)
- Try Dueling DQN on Atari games
- Explore **Noisy Nets** for better exploration
- Read the original paper: "Dueling Network Architectures for Deep RL" (Wang et al., 2016)